# SPOTIFY Data Analysis

# Name of student
BENYAHIA Hikmet

# Problem Understanding

The music industry is growing faster than ever. Each year, new platforms and mediums skyrocket to stand out, make a name for themselves, and reshape the way audiences connect with artists. At the same time, new technologies put creative tools in the hands of people who were previously unable to access them. Obviously, this industry is growing rapidly and becoming more competitive. Companies are having to differentiate themselves by providing unique, artist-focused content or developing different pricing models. Spotify, founded in 2006, has become the most popular streaming platform worldwide. We will use the data set from this platform to analyze song trends of singers. From there, it is possible to suggest solutions to improve the popularity of singers, which in turn will help them be more competitive and successful in the music industry.

# Import libraries

In [ ]:
!pip install pyspark

In [ ]:
import numpy as np
import pandas as pd
from wordcloud import WordCloud

import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.types as t
import pyspark.sql.functions as f

# Import Data

In [ ]:
data = pd.read_csv('/charts.csv')

Get the shape of data frame:

In [ ]:
data.shape

The number of countries:

In [ ]:
len(data['region'].unique())

# Get the visualization of artist popularity

In [ ]:
def visualize_word_counts(counts):
    wc = WordCloud(max_font_size=130, min_font_size=25, colormap='tab20', background_color='white', 
                   prefer_horizontal=.95, width=2100, height=700, random_state=0)
    cloud = wc.generate_from_frequencies(counts)
    plt.figure(figsize=(18,15))
    plt.imshow(cloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

In [ ]:
counts = data['artist'].value_counts()

This counts and sorts all observations of songs by artists in 70 countries in Spotify's Top200:

The top 10 most popular artists:

In [ ]:
counts.head(10)

The top 10 least popular artists:

In [ ]:
counts.tail(10)

In [ ]:
counts.loc['Aspova',]

In [ ]:
visualize_word_counts(counts)

We are going to compare the trend of songs between the most popular singer Ed Sheeran and the least popular singer Aspova to see the difference. From there we will try to suggest ways to improve their popularity.

# Streams by region
Which region's Top200 list is streamed the most? The result will be somewhat related to the region's population.

In [ ]:
# compute total streams by region
streams = data.groupby('region')['streams'].sum().reset_index()

# compute percent stream
streams['percent_streams'] = streams['streams']/streams['streams'].sum()

# rename regions with very little streams (< .01 %) as 'Other'
streams['region'] = streams.apply(lambda x: x['region'] if x['percent_streams'] >= .01 else 'Other', axis=1)

# we need another groupby because there are multiple regions with name 'Other'
streams = streams.groupby('region')['percent_streams'].sum().reset_index().round(3).sort_values(by='percent_streams')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.pie(x=streams['percent_streams'], labels=streams['region'], autopct='%.1f%%')
ax.set_title('Streams by region')
plt.tight_layout()

Now we try to use Spark and SQL to explore the trends in the dataset:

A SparkSession can be used create:
* DataFrame
* Register DataFrame as tables
* Execute SQL over tables
* Cache tables
* Read parquet files

To create a SparkSession, use the following builder pattern:

In [ ]:
spark = (SparkSession.builder.config("spark.driver.memory","4g").config("spark.driver.maxResultSize", "4g").getOrCreate())

Now, we need to import our data again. We are specifying the desire to use dataset csv format.
* “header”: true if the first row in the dataset are column names.
* “inferSchema” instructs Spark to attempt to infer the schema of the CSV

In [ ]:
# import dataframe by spark
df = spark.read.csv(path='../input/spotify-charts/charts.csv', inferSchema=True, header=True)

We cannot perform our analysis with all the datatype string. Thus here we parse the datatypes using the format below.

Datatypes supported by Spark: https://spark.apache.org/docs/3.0.0-preview2/sql-ref-datatypes.html

We convert 'rank', 'date' and 'streams' in the original dataset to data types in Spark.

In [ ]:
# setting the schema

df = df.withColumn("rank", f.col("rank").cast(t.IntegerType())).withColumn("date", f.col("date").cast(t.DateType())).withColumn("streams", f.col("streams").cast(t.IntegerType()))

Take a look at the DataFrame:

In [ ]:
df

To perform analysis, we will be using spark sql library. For that, we create a temp table.

The reason to use the registerTempTable( tableName ) method for a DataFrame, is so that in addition to being able to use the Spark-provided methods of a DataFrame.

Using this, we can also issue SQL queries via the sqlContext. sql( sqlQuery ) method, that use that DataFrame as an SQL table.

In [ ]:
# Register a temp table
df.registerTempTable("charts")

# Get the range of timeline of the data

In [ ]:
spark.sql('''
SELECT MIN(date) begin, MAX(date) end 
FROM charts 
WHERE chart = 'top200';
''').toPandas()

# View the head of the chart

In [ ]:
spark.sql('''
SELECT * 
FROM charts 
WHERE chart = 'top200' 
LIMIT 10;
''').toPandas()

# Get the total number of observations by all the singers in TOP 200 chart

In [ ]:
spark.sql('''
SELECT COUNT(*) NoOfObservationsTop200
FROM charts 
WHERE chart = 'top200';
''').toPandas().head(10)

=> Over 20 million observations in Top 200 over 4 years (1/1/2017 - 12/31/2021)

# Get the total number of songs by Ed Sheeran & Aspova appeared in Top 200

## Ed Sheeran

In [ ]:
spark.sql('''
SELECT COUNT(DISTINCT title) NoOfSongs 
FROM charts 
WHERE artist LIKE '%Ed Sheeran%' AND chart = 'top200';
''').toPandas().head(10)

## Aspova

In [ ]:
spark.sql('''
SELECT COUNT(DISTINCT title) NoOfSongs 
FROM charts 
WHERE artist LIKE '%Aspova%' AND chart = 'top200';
''').toPandas().head(10)

=> Ed Sheeran's songs appear in the Top 200 6 times more than Aspova's songs in the Top 200

# Count the number of times Ed Sheeran & Aspova appeared in the TOP 200 chart

## Ed Sheeran

In [ ]:
spark.sql('''
SELECT COUNT(*) NoInTop200 
FROM charts 
WHERE artist LIKE '%Ed Sheeran%' AND chart = 'top200';
''').toPandas().head(10)

## Aspova

In [ ]:
spark.sql('''
SELECT COUNT(*) NoInTop200 
FROM charts 
WHERE artist LIKE '%Aspova%' AND chart = 'top200';
''').toPandas().head(10)

=> Ed Sheeran appeared in the Top 200 300 times more than Aspova

# Count the number of times Ed Sheeran and Aspova appeared in the Top 200 while collaborating with other singers

## Ed Sheeran

In [ ]:
spark.sql('''
SELECT artist, COUNT(artist) count 
FROM charts 
WHERE artist LIKE '%Ed Sheeran%' AND chart = 'top200' 
GROUP BY artist 
ORDER BY count DESC;
''').toPandas().head(10)

## Aspova

In [ ]:
spark.sql('''
SELECT artist, COUNT(artist) count 
FROM charts 
WHERE artist LIKE '%Aspova%' AND chart = 'top200' 
GROUP BY artist 
ORDER BY count DESC;
''').toPandas().head(10)

=> Ed Sheeran and Aspova are both more successful solo.

=> Ed Sheeran's songs are 500 times more popular than Aspova's songs.

# Show the top 10 most popular songs of Ed Sheeran and Aspova

## Ed Sheeran

In [ ]:
spark.sql('''
SELECT title, SUM(streams) streams 
FROM charts 
WHERE artist LIKE '%Ed Sheeran%' 
AND streams IS NOT NULL 
GROUP BY title 
ORDER BY streams DESC 
LIMIT 10;
''').toPandas().head(10)

## Aspova

In [ ]:
spark.sql('''
SELECT title, SUM(streams) streams 
FROM charts 
WHERE artist LIKE '%Aspova%' 
AND streams IS NOT NULL 
GROUP BY title 
ORDER BY streams DESC 
LIMIT 10;
''').toPandas().head(10)

=> Shape of You of Ed Sheeran has 5 billion streams, while Eskimiş Senelere of Aspova only has 37 million streams

=> The difference between the top song of Ed Sheeran and Aspova is 135 times

# Count the number of times top songs of Ed Sheeran & Aspova has appeared in top 200

## Ed Sheeran

In [ ]:
spark.sql('''
SELECT title, COUNT(title) AS count 
FROM charts 
WHERE artist LIKE '%Ed Sheeran%' 
AND chart = 'top200' 
GROUP BY title 
ORDER BY count DESC;
''').toPandas().head(10)

## Aspova

In [ ]:
spark.sql('''
SELECT title, COUNT(title) AS count
FROM charts 
WHERE artist LIKE '%Aspova%' 
AND chart = 'top200' 
GROUP BY title 
ORDER BY count DESC;
''').toPandas().head(10)

=> Shape of You of Ed has appeared in the Top 200 100 times more than Aspova's Eskimiş Senelere

=> It also the number of days Top songs of Ed Sheeran & Aspova were on the Top 200 chart

=> The number of days Top songs of Ed Sheeran has appeared in the Top 200 100 times more than Aspova's Eskimiş Senelere

=> The top songs of Ed Sheeran meet the demands of audience better than that of Aspova => They last longer on the Spotify Top 200

=> Audience prefers pop music of Ed Sheeran rather than rap music of Aspova

# Get the highest ranks the songs of Ed Sheeran and Aspova have attained

# Get the number of times they have been in that position

## Ed Sheeran

In [ ]:
spark.sql('''
SELECT title, MIN(rank) rank, COUNT(rank) count 
FROM charts 
WHERE artist LIKE '%Ed Sheeran%'
AND chart = 'top200' 
AND rank <= 10 
GROUP BY title
ORDER BY rank ASC;
''').toPandas().head(20)

## Aspova

In [ ]:
spark.sql('''
SELECT title, MIN(rank) rank, COUNT(rank) count 
FROM charts 
WHERE artist LIKE '%Aspova%' 
AND chart = 'top200' 
AND rank <= 10 
GROUP BY title
ORDER BY rank ASC;
''').toPandas().head(20)

=> Ed Sheeran has 9 songs in rank 1. Aspova only has 1 song in rank 1.

=> Ed Sheeran's Shape of You is the most loved song and also has the highest chart for a long time worldwide. Aspova's Eskimis Senelere is his most popular song, but it only peaked at number 10 on one occasion.

Susamam is a collaboration song of Aspova and it ranks 1 => Collaboration is better for Aspova to be more popular than solo.

=> Shape of You is Ed Sheeran's most successful song. It stayed at #1 for 6 times longer than the second song Castle of the Hill

# Count the number times Ed Sheeran and Aspova have been on the 1st rank

## Ed Sheeran

In [ ]:
spark.sql('''
SELECT COUNT(*) NoOfRank1 
FROM charts 
WHERE artist LIKE '%Ed Sheeran%' 
AND chart = 'top200' 
AND rank = 1;
''').toPandas().head(10)

## Aspova

In [ ]:
spark.sql('''
SELECT COUNT(*) NoOfRank1 
FROM charts 
WHERE artist LIKE '%Aspova%' 
AND chart = 'top200' 
AND rank = 1;
''').toPandas().head(10)

=> Ed Sheeran has been in rank 1 400 times more than Aspova

# Calculate the highest, lowest and the mean rank of the songs of Ed Sheeran and Aspova

## Ed Sheeran

In [ ]:
spark.sql('''
SELECT Title, MIN(rank) Highest, MAX(rank) Lowest, AVG(rank) Avg 
FROM charts 
WHERE artist like '%Ed Sheeran%' 
AND chart='top200' 
GROUP BY title 
ORDER BY Highest;
''').toPandas().head(10)

## Aspova

In [ ]:
spark.sql('''
SELECT Title, MIN(rank) Highest, MAX(rank) Lowest, AVG(rank) Avg 
FROM charts 
WHERE artist like '%Aspova%' 
AND chart='top200' 
GROUP BY title 
ORDER BY Highest;
''').toPandas().head(10)

=> Ed Sheeran has much more songs in rank 1 than Aspova

=> Most songs of Aspova are around rank 50

# Show the trends of top songs of Ed Sheeran and Aspova over time

## Ed Sheeran

In [ ]:
q = spark.sql('''
SELECT title, YEAR(date) year, date, rank, region 
FROM charts 
WHERE title IN ('Shape of You','Castle on the Hill','Perfect') 
AND chart='top200'
ORDER BY rank ASC;''').toPandas()

In [ ]:
fig, axes = plt.subplots(figsize=(20,7))
sns.lineplot('date', 'rank', data=q, hue='title', ci=None).set_title('Trends in Top 200')

## Aspova

In [ ]:
p = spark.sql('''
SELECT title, YEAR(date) year, date, rank, region 
FROM charts 
WHERE title IN ('Eskimiş Senelere','Susamam','Suç') 
AND chart='top200'
ORDER BY rank ASC;''').toPandas()

In [ ]:
fig, axes = plt.subplots(figsize=(20,7))
sns.lineplot('date', 'rank', data=p, hue='title', ci=None).set_title('Trends in Top 200')

=> Songs of Ed Sheeran began in rank 1 and slowly decreased to rank 200, but still in the Top 200 chart

=> Songs of Aspova began in rank below 100, then quickly increased to rank 1, then slowly decreased to rank 200 and then were out of Top 200 chart